## Alles good doen

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BigqueryExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load data from BigQuery.


In [3]:
df_individuals = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.individuals")    # project_id.datatset.tablename. Use your project id
df_individuals.printSchema()

root
 |-- ID: long (nullable = true)
 |-- age: long (nullable = true)
 |-- has_spouse: boolean (nullable = true)
 |-- spouse_ID: long (nullable = true)
 |-- gross_salary: long (nullable = true)
 |-- has_student_loan: boolean (nullable = true)
 |-- student_loan_amount: long (nullable = true)
 |-- has_general_loan: boolean (nullable = true)
 |-- general_loan_amount: long (nullable = true)
 |-- has_alimony: boolean (nullable = true)
 |-- alimony_amount: long (nullable = true)



In [4]:
df_spouse = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.spouse")   
df_spouse.printSchema()

root
 |-- spouse_ID: long (nullable = true)
 |-- age: long (nullable = true)
 |-- gross_salary: long (nullable = true)
 |-- has_student_loan: boolean (nullable = true)
 |-- student_loan_amount: long (nullable = true)
 |-- has_general_loan: boolean (nullable = true)
 |-- general_loan_amount: long (nullable = true)
 |-- has_alimony: boolean (nullable = true)
 |-- alimony_amount: long (nullable = true)



In [15]:
df_toetsinkomen = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.toetsinkomen")   
df_toetsinkomen.show()

+------------+----------+
|Toetsinkomen|Toetsrente|
+------------+----------+
|           0|      17.5|
|       19500|      17.5|
|       20000|      18.5|
|       20050|      19.5|
|       21000|      20.5|
|       21500|      21.0|
|       22000|      22.0|
|       22500|      23.5|
|       23000|      24.5|
|       23500|      25.0|
|       24000|      25.5|
|       25000|      26.5|
|       26000|      27.0|
|       28000|      27.5|
|       55000|      28.0|
|       58000|      28.5|
|       61000|      29.0|
|       63000|      29.5|
|       65000|      30.0|
|       68000|      30.5|
+------------+----------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import col, when


df_comb = df_individuals.alias("ind").join(df_spouse.alias("sp"), col("ind.ID") ==  col("sp.spouse_ID"),"inner")

In [9]:
df = df_comb
# Convert necessary columns to appropriate data types (e.g., from string to integer/boolean)
df = df.withColumn("ind.gross_salary", col("ind.gross_salary").cast("int"))
df = df.withColumn("ind.alimony_amount", col("ind.alimony_amount").cast("int"))
df = df.withColumn("ind.has_spouse", col("ind.has_spouse") == "True")
df = df.withColumn("sp.gross_salary", col("sp.gross_salary").cast("int"))
df = df.withColumn("sp.alimony_amount", col("sp.alimony_amount").cast("int"))

# Apply the formula
df = df.withColumn("house_spendable_income", 
                   when(col("ind.has_spouse") == False, col("ind.gross_salary") - 12 * col("ind.alimony_amount"))
                   .otherwise(col("ind.gross_salary") - 12 * col("ind.alimony_amount")+ 1/3 * (col("sp.gross_salary")-12 * col("sp.alimony_amount"))))

df = df.withColumn("house_spendable_income", col("house_spendable_income").cast("int"))
#df_comb.select("ind.student_loan_amount", "sp.student_loan_amount").show()
df.select("ind.gross_salary","ind.alimony_amount","ind.has_spouse","sp.gross_salary","sp.alimony_amount", "house_spendable_income").show()



+------------+--------------+----------+------------+--------------+----------------------+
|gross_salary|alimony_amount|has_spouse|gross_salary|alimony_amount|house_spendable_income|
+------------+--------------+----------+------------+--------------+----------------------+
|       75526|             0|      true|       59122|             0|                 95233|
|      134600|          1200|     false|      101412|             0|                120200|
|      144798|             0|      true|      149733|             0|                194709|
|      150580|             0|      true|       47582|             0|                166440|
|      186763|             0|     false|      181201|             0|                186763|
|       59116|             0|      true|       38986|             0|                 72111|
|      189883|          3697|      true|      143379|             0|                193312|
|      157499|             0|      true|       65001|             0|            

## Step 2 get for each row the corresponding toetsinkomen rente and add as column to df

In [16]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
def get_interest(inkomen):
    df = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.toetsinkomen") 
    rente = df.filter(df['Toetsinkomen'] < inkomen).orderBy(df['Toetsinkomen'].desc()).first()
    return rente['Toetsrente']

custom_udf = udf(get_interest, FloatType())

# Apply the mapping function using withColumn to create a new column "Toetsrente"
df_income_w_interest = df.withColumn(
    "Toetsrente", 
    custom_udf(col("house_spendable_income"))
)
df_income_w_interest.show()

Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/context.py", line 466, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [17]:
spark.stop()